In [4]:
'''
#sample huggingface model
https://huggingface.co/docs/transformers/model_doc/gpt2
#data for training is at git location
https://github.com/ryanzhumich/AESLC
#Some models to read throught o use -- mentor provided
https://arxiv.org/abs/2310.06825
https://arxiv.org/abs/2307.09288
https://arxiv.org/abs/2309.16609 '''

'\n#sample huggingface model\nhttps://huggingface.co/docs/transformers/model_doc/gpt2\n#data for training is at git location\nhttps://github.com/ryanzhumich/AESLC\n#Some models to read throught o use -- mentor provided\nhttps://arxiv.org/abs/2310.06825\nhttps://arxiv.org/abs/2307.09288\nhttps://arxiv.org/abs/2309.16609 '

In [5]:
!pip install -U transformers[torch]
!pip install -U huggingface_hub
! pip install evaluate
!pip install sacrebleu
!pip install rouge_score
!pip install -U nltk
#!pip install bitsandbytes
!pip install tqdm


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5a11be3a29c9587f0e15c68841fe43974adb7cc66850feed2f1c3f2826563bea
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
#clone the data from git
!git clone https://github.com/ryanzhumich/AESLC.git

fatal: destination path 'AESLC' already exists and is not an empty directory.


In [8]:
import numpy as np
import pandas as pd
import os
import random
import shutil

In [8]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, DataCollatorForLanguageModeling #Seq2SeqTrainingArguments, Seq2SeqTrainer
import accelerate
import evaluate
#import bitsandbytes
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names
import re


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [2]:
#define a function for subset data copying
def subsetFileCopy(sourceDir, destDir, countToCopy):
  #remove and recreate destDir directory
  if os.path.exists(destDir):
    shutil.rmtree(destDir)
  os.makedirs(destDir)
  #Read Source directory to randomly select few files and copy
  for dirname, _, filenames in os.walk(sourceDir):
      #count how many files we are writing
      count = 0
      for count in range(countToCopy):
        filename = random.choice(filenames)
        source = os.path.join(dirname, filename)
        destination = os.path.join(destDir,filename)
        shutil.copy(source, destination)
        count += 1

In [3]:
# Copy subset train data to train subset directory
train_dir        = '/content/AESLC/enron_subject_line/train'
train_subset_dir = '/content/AESLC/enron_subject_line/train_subset'

val_dir          = '/content/AESLC/enron_subject_line/dev'
val_subset_dir   = '/content/AESLC/enron_subject_line/dev_subset'

test_dir         = '/content/AESLC/enron_subject_line/test'
test_subset_dir  = '/content/AESLC/enron_subject_line/test_subset'

base_dir = '/content/AESLC/enron_subject_line/'

In [10]:
def mergeEmailFilesIntoCSV(TargetCSVFilename, sourceDirPath):
    filename = TargetCSVFilename
    dirpath = sourceDirPath
    output = base_dir+filename
    #Remove if csv already exists
    if os.path.exists(output):
        os.remove(output)

    csvout_lst = []
    files = [os.path.join(dirpath, fname) for fname in os.listdir(dirpath)]

    for filename in sorted(files):
        #open each email file and process to seperate email body and subject with delimiter @subject
        #data = pd.read_csv(filename, sep="@subject\n", header=None)
        with open(filename, 'r') as fin:
          lines = [line.strip('\n')  for line in fin]
          #reset to use for next file
          subIndex = 0
          email=''
          subject=''
          row_num = 0
          #find the delimiting line index
          for i, t in enumerate(lines):
            if t =='@subject':
              subIndex = i
          #write to df columns based on delimiter index
          for i, text in enumerate(lines):
            if i<subIndex :
              email = email+text+' '
            elif i>subIndex:
              subject = subject+text+' '
          new_row =  pd.DataFrame({'Row_Num': [row_num],'Email':[email], 'Subject':[subject]})
          row_num += 1
          table = pd.pivot_table(new_row, index =['Row_Num','Email', 'Subject'])
          csvout_lst.append(table)
    pd.concat(csvout_lst).to_csv(output)

In [11]:
def mergeEmailFilesIntoCSV_alongWithAnnotations(TargetCSVFilename, sourceDirPath):
    filename = TargetCSVFilename
    dirpath = sourceDirPath
    output = base_dir+filename
    #Remove if csv already exists
    if os.path.exists(output):
        os.remove(output)

    csvout_lst = []
    files = [os.path.join(dirpath, fname) for fname in os.listdir(dirpath)]

    for filename in sorted(files):
        #open each email file and process to seperate email body and subject with delimiter @subject
        #data = pd.read_csv(filename, sep="@subject\n", header=None)
        with open(filename, 'r') as fin:
          lines = [line.strip('\n')  for line in fin]
          #reset to use for next file
          email=''
          subject=''
          ann0=''
          ann1=''
          ann2=''
          subIndex = 0
          ann0Index = 0
          ann1Index = 0
          ann2Index = 0
          row_num = 0

          #find the delimiting line index
          for i, t in enumerate(lines):
            if t =='@subject':
              subIndex = i
            elif t == '@ann0':
              ann0Index = i
            elif t == '@ann1':
              ann1Index = i
            elif t == '@ann2':
              ann2Index = i

          #write to df columns based on delimiter index
          for i, text in enumerate(lines):
            if i<subIndex :
              email = email+text+' '
            elif i>subIndex and i<ann0Index :
              subject = subject+text+' '
            elif i>ann0Index and i<ann1Index:
              ann0 = ann0+text+' '
            elif i>ann1Index and i<ann2Index:
              ann1 = ann1+text+' '
            elif i>ann2Index:
              ann2 = ann2+text+' '
          new_row =  pd.DataFrame({'Row_Num': [row_num], 'Email':[email], 'Subject':[subject], 'Ann0':[ann0], 'Ann1':[ann1], 'Ann2':[ann2]})
          row_num += 1
          table = pd.pivot_table(new_row, index =['Row_Num', 'Email', 'Subject', 'Ann0', 'Ann1', 'Ann2'])
          csvout_lst.append(table)
    pd.concat(csvout_lst).to_csv(output)

In [12]:
#=====================SUBSET FILES====================================#
#call function to copy subset of data
subsetFileCopy(train_dir, train_subset_dir, 1000)
subsetFileCopy(val_dir, val_subset_dir, 10)
subsetFileCopy(test_dir, test_subset_dir, 10)

#call function to make csv from subset Train email files
mergeEmailFilesIntoCSV('train_subset.csv', train_subset_dir)
mergeEmailFilesIntoCSV_alongWithAnnotations('val_subset.csv', val_subset_dir)
mergeEmailFilesIntoCSV_alongWithAnnotations('test_subset.csv', test_subset_dir)

#This is train on subset data folders
df = pd.read_csv(base_dir+'train_subset.csv')
df_val = pd.read_csv(base_dir+'val_subset.csv')
df_test = pd.read_csv(base_dir+'test_subset.csv')

In [17]:
#========================================FULL FILES=================================#

#call function to make csv from subset Train email files
mergeEmailFilesIntoCSV('train.csv', train_dir)
mergeEmailFilesIntoCSV_alongWithAnnotations('val.csv', val_dir)
mergeEmailFilesIntoCSV_alongWithAnnotations('test.csv', test_dir)

#This is train on whole dataset folders
df = pd.read_csv(base_dir+'train.csv')
df_val = pd.read_csv(base_dir+'val.csv')
df_test = pd.read_csv(base_dir+'test.csv')

In [14]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [15]:
sacrebleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

NameError: name 'evaluate' is not defined

In [ ]:
torch.manual_seed(42)


In [ ]:
#make a directory
if not os.path.exists('/content/working/output'):
    os.makedirs('/content/working/output')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq,Trainer, TrainingArguments

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model.resize_token_embeddings(len(tokenizer))

In [ ]:
'''tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>', sep_token='<|sep|>')
model = GPT2LMHeadModel.from_pretrained('distilgpt2').cuda()
model.resize_token_embeddings(len(tokenizer))'''

In [ ]:
datacollator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model)


In [ ]:
datacollator

In [ ]:
class EmailSubjectDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
            encodings_dict = tokenizer('<|startoftext|>' + row["Email"] + '<|sep|>' + row["Subject"] + '<|endoftext|>', truncation=True, max_length=250, padding="max_length", return_tensors='pt')
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):

       return self.input_ids[idx]

In [ ]:
class ValEmailSubjectDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["Email"] + '<|sep|>', truncation=True, max_length=250, padding="max_length", return_tensors='pt')
          if((encodings_dict['input_ids'][0][249] != torch.Tensor(np.array([50259]))) and (encodings_dict['input_ids'][0][249] != torch.Tensor(np.array([50258])))):
            encodings_dict['input_ids'][0] = torch.cat((encodings_dict['input_ids'][0][np.r_[:249]],torch.Tensor(np.array([50258]))),0)
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, idx):
        return self.input_ids[idx]

In [ ]:
class TestEmailSubjectDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for index, row in data.iterrows():
          encodings_dict = tokenizer('<|startoftext|>' + row["Email"] + '<|sep|>', truncation=True, max_length=250, return_tensors='pt')
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, idx):
        return self.input_ids[idx]

In [ ]:
train_dataset = EmailSubjectDataset(df, tokenizer)
val_dataset = ValEmailSubjectDataset(df_val,tokenizer)
test_dataset = TestEmailSubjectDataset(df_test,tokenizer)

In [ ]:
#train_dataset.attn_masks

In [ ]:
training_args = TrainingArguments(
    output_dir="/Content/working/output",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    #eval_steps = 2,
    eval_steps = 400,
    save_steps=800,
    warmup_steps=500,
    load_best_model_at_end=True,
    #evaluation_strategy="steps",
    #save_strategy = "steps",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    weight_decay=0.01,
    #metric_for_best_model = "rougeL",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True
    )


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import tqdm
from tqdm import tqdm # Import the tqdm function

def compute_metrics(eval_pred, eval_dataset, df):
    decoded_preds = []
    references = [df['Subject'], df['Ann0'], df['Ann1'], df['Ann2']]
    refs = []


    '''for i in range(len(eval_dataset)):
        sample = eval_dataset[i]
        # Assuming sample is a dictionary-like object with 'Row_Num' key
        Row_Num = sample.get('Row_Num', None)
        if Row_Num is None:
            print(f"Warning: Sample at index {i} does not have 'input_ids' key. Skipping.")
            continue
        # Convert Row_Num to a tensor for comparison
        Row_Num_tensor = torch.tensor(Row_Num)
        # Filter Row_Num_tensor based on the length of eval_dataset
        temp_input = Row_Num_tensor[Row_Num_tensor != len(eval_dataset)]

        temp_input = temp_input[None, :]
        metric_outputs = model.generate(temp_input.cuda(), attention_mask=eval_dataset.attn_masks,min_new_tokens = 4, max_new_tokens = 12, num_beams=5, early_stopping=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        decoded_preds.append(tokenizer.decode(metric_outputs[0]))
'''
    for idx, pred in tqdm(enumerate(eval_pred.predictions)):
        temp_input = torch.tensor(pred).long()
        # Access attention mask for the current sample
        attention_mask = eval_dataset[idx]['attention_mask']
        temp_input = temp_input[None, :]
        metric_outputs = model.generate(torch.tensor(temp_input).cuda(),
                                        attention_mask=torch.tensor(attention_mask).cuda(),
                                        min_new_tokens = 4,
                                        max_new_tokens = 12, num_beams=5,
                                        early_stopping=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        decoded_preds.append(tokenizer.decode(metric_outputs[0]))

    final_preds =[]
    for j in range(len(decoded_preds)):
        lst = decoded_preds[j].split('<|sep|>')
        if (len(lst) >= 2):
            final_preds.append(lst[1].replace("<|endoftext|>",""))
        temp_refs = []
        for k in range(len(references)):
            temp_refs.append(references[k][j])
        refs.append(temp_refs)

    results_sacrebleu = sacrebleu.compute(predictions=final_preds, references=refs, lowercase = True)

    results_rouge = rouge.compute(predictions=final_preds, references=refs)

    results_meteor = meteor.compute(predictions=final_preds, references=refs)

    return {'bleu': results_sacrebleu['score'], 'rouge1' : results_rouge['rouge1'], 'rouge2' : results_rouge['rouge2'], 'rougeL' : results_rouge['rougeL'], 'meteor' : results_meteor['meteor']}

In [ ]:
'''def compute_metrics(eval_pred, eval_dataset, df):
    decoded_preds = []
    # Assuming 'eval_pred.predictions' is the model output
    for sample_input in eval_pred.predictions:
        temp_input = torch.tensor(sample_input) # Convert to tensor if it's not already
        temp_input = temp_input[temp_input != tokenizer.pad_token_id] # Remove padding tokens
        if temp_input.numel() > 0: # Check if tensor is not empty
            temp_input = temp_input[None, :]  # Add a batch dimension if necessary
            metric_outputs = model.generate(temp_input.cuda(), min_new_tokens=4, max_new_tokens=12,
                                           num_beams=5, early_stopping=True, num_return_sequences=1,
                                           pad_token_id=tokenizer.eos_token_id)
            decoded_preds.append(tokenizer.decode(metric_outputs[0]))
        else:
            decoded_preds.append("") # Handle empty tensors

    final_preds =[]
    for j in range(len(decoded_preds)):
        lst = decoded_preds[j].split('<|sep|>')
        if (len(lst) >= 2):
            final_preds.append(lst[1].replace("<|endoftext|>",""))
        temp_refs = []
        for k in range(len(references)):
            temp_refs.append(references[k][j])
        refs.append(temp_refs)

    results_sacrebleu = sacrebleu.compute(predictions=final_preds, references=refs, lowercase = True)

    results_rouge = rouge.compute(predictions=final_preds, references=refs)

    results_meteor = meteor.compute(predictions=final_preds, references=refs)

    return {'bleu': results_sacrebleu['score'], 'rouge1' : results_rouge['rouge1'], 'rouge2' : results_rouge['rouge2'], 'rougeL' : results_rouge['rougeL'], 'meteor' : results_meteor['meteor']}
'''

In [ ]:
'''def compute_metrics(eval_pred, eval_dataset, df):
    logits, labels = eval_pred
    # Handle the case where logits is a tuple (e.g., from evaluation_loop)
    if isinstance(logits, tuple):
        logits = logits[0]  # Extract the logits tensor from the tuple

    pred_ids = torch.argmax(torch.Tensor(logits), dim=-1)  # Get predicted token IDs
    # Ensure the shape of pred_ids and labels is correct for the T5 model
    pred_ids = pred_ids.view(-1, pred_ids.size(-1))
    # Check if labels is a tensor before reshaping
    if isinstance(labels, torch.Tensor):
        labels = labels.view(-1, labels.size(-1))  # Flatten labels as well to maintain consistency
    else:
        print("Warning: labels is not a torch.Tensor. Skipping reshaping.")

    # Convert predictions and labels to lists
    pred_ids = pred_ids.tolist()
    labels = labels.tolist()

    # ... rest of your compute_metrics function ...

    non_padded_preds = []
    for pred in pred_ids:
        temp_input = torch.tensor([i for i in pred if i != -100]) # Filter out padding tokens
        if temp_input.numel() > 0:
            temp_input = temp_input.unsqueeze(0).cuda()
            decoded_pred = tokenizer.decode(temp_input[0], skip_special_tokens=True) # Decode token IDs to text
            non_padded_preds.append(decoded_pred)
        else:
            non_padded_preds.append("") # Handle empty predictions

    # Replace decoded_preds with non-padded versions
    decoded_preds = non_padded_preds

    final_preds =[]
    for j in range(len(decoded_preds)):
        lst = decoded_preds[j].split('<|sep|>')
        if (len(lst) >= 2):
            final_preds.append(lst[1].replace("<|endoftext|>",""))
        temp_refs = []
        for k in range(len(references)):
            temp_refs.append(references[k][j])
        refs.append(temp_refs)

    results_sacrebleu = sacrebleu.compute(predictions=final_preds, references=refs, lowercase = True)

    results_rouge = rouge.compute(predictions=final_preds, references=refs)

    results_meteor = meteor.compute(predictions=final_preds, references=refs)

    return {'bleu': results_sacrebleu['score'], 'rouge1' : results_rouge['rouge1'], 'rouge2' : results_rouge['rouge2'], 'rougeL' : results_rouge['rougeL'], 'meteor' : results_meteor['meteor']}


 result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract the Rouge2 score
    rouge2_score = result['rouge2'].mid.fmeasure
    return {"rouge2": rouge2_score}'''

In [ ]:
'''
# Tokenization
from datasets import Dataset
# Assuming 'df' contains your training data
train_dataset = Dataset.from_pandas(df)
# Assuming 'df_val' contains your validation data
val_dataset = Dataset.from_pandas(df_val)
#assuming 'df_test' contains your test data
test_dataset = Dataset.from_pandas(df_test)

def preprocess_data(examples):
    inputs = ["generate subject line: " + email for email in examples['Email']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Subject'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True) '''

In [ ]:
'''
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # Handle the case where logits is a tuple (e.g., from evaluation_loop)
    if isinstance(logits, tuple):
        logits = logits[0]  # Extract the logits tensor from the tuple

    pred_ids = torch.argmax(logits, dim=-1)
    # Ensure the shape of pred_ids and labels is correct for the T5 model
    pred_ids = pred_ids.view(-1, pred_ids.size(-1))
    labels = labels.view(-1, labels.size(-1))
    return pred_ids, labels
    '''

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # Handle the case where logits is a tuple (e.g., from evaluation_loop)
    if isinstance(logits, tuple):
        logits = logits[0]  # Extract the logits tensor from the tuple

    pred_ids = torch.argmax(logits, dim=-1)
    # Ensure the shape of pred_ids and labels is correct for the T5 model
    pred_ids = pred_ids.view(-1, pred_ids.size(-1))
    labels = labels.view(-1, labels.size(-1)) # Flatten labels as well to maintain consistency
    return pred_ids, labels  # Return predicted token IDs and labels

In [ ]:
'''
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # Handle the case where logits is a tuple (e.g., from evaluation_loop)
    if isinstance(logits, tuple):
        logits = torch.Tensor(logits[0])  # Extract the logits tensor from the tuple

    pred_ids = torch.argmax(logits, dim=-1)  # Get predicted token IDs
    # Ensure the shape of pred_ids and labels is correct for the T5 model
    pred_ids = pred_ids.view(-1, pred_ids.size(-1))
    labels = labels.view(-1, labels.size(-1)) # Flatten labels as well to maintain consistency
    return pred_ids, labels  # Return predicted token IDs and labels
    '''

In [ ]:
import torch

# Check for available CUDA devices
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Set the environment variable to enable more verbose CUDA error reporting
!export CUDA_LAUNCH_BLOCKING=1

# Commenting out the line below to see if the error persists
# torch.cuda.empty_cache()

trainer = Trainer(
    model=model.to(device),  # Move the model to the appropriate device
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=datacollator,
    compute_metrics=lambda pred: compute_metrics(pred, val_dataset, df_val),
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

In [ ]:
# import wandb
# wandb.login(key = "{your token here}")

In [ ]:
trainer.train()


In [ ]:
trainer.save_model()


In [ ]:
#trainedmodel = GPT2LMHeadModel.from_pretrained("/Content/input/trainedmodel").cuda()
#trainedmodel.resize_token_embeddings(len(tokenizer))

In [ ]:
#test_dataset = TestEmailSubjectDataset(df_test,tokenizer)
#len(test_dataset)

In [ ]:
temp_output = []
for i, sample_input in enumerate(test_dataset):
    if(len(sample_input[0]) == 250 and sample_input[0][249] != torch.Tensor(np.array([50258]))):
        sample_input[0] = torch.cat((sample_input[0][np.r_[:249]],torch.Tensor(np.array([50258]))),0)
    sample_output = model.generate(sample_input.cuda(), min_new_tokens = 4, max_new_tokens = 12, num_beams=5, early_stopping=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    temp_output.append(tokenizer.decode(sample_output[0]))
final_output =[]
for j in range(len(temp_output)):
    lst = temp_output[j].split('<|sep|>')
    if (len(lst) >= 2):
        final_output.append(lst[1].replace("<|endoftext|>",""))
    else:
        final_output.append("")

In [ ]:
df_test["Generated"] = final_output
df_test.to_csv('/Content/working/Generated.csv')

In [ ]:
#df_test = pd.read_csv("/Content/input/generatedoutput/Generated.csv")


In [ ]:
pred = df_test['Generated']
ref = [df_test['Subject'], df_test['Ann0'], df_test['Ann1'], df_test['Ann2']]

In [ ]:

def score_evaluate(predictions, references):
    preds = []
    refs = []
    for i in range(len(predictions)):
        preds.append(predictions[i])
        temp_refs = []
        for j in range(len(references)):
            temp_refs.append(references[j][i])
        refs.append(temp_refs)
    results_sacrebleu = sacrebleu.compute(predictions=preds, references=refs, lowercase = True)
    print("Bleu Score : " + str(results_sacrebleu['score']))

    results_rouge = rouge.compute(predictions=preds, references=refs)
    print("Rouge1 Score : " + str(results_rouge['rouge1']))
    print("Rouge2 Score : " + str(results_rouge['rouge2']))
    print("RougeL Score : " + str(results_rouge['rougeL']))

    results_meteor = meteor.compute(predictions=preds, references=refs)
    print("Meteor Score : " + str(results_meteor['meteor']))

In [ ]:
score_evaluate(pred,ref)